# **Problema 2 - Generacion de texto con RNN.**

**Descripción:**

En el siguiente problema, se presenta un conjunto de datos correspondientes a escritos de Shakespear. El objetivo del problema es crear un modelo capaz de generar texto con dialecto de época y escritura en verso y prosa.

**Dataset:**

El dataset incluye 40000 líneas de distintos escritos de Shakespear. Sólo utilizaremos el dataset como un cuerpo de texto para entrenar un modelo recurrente de generación de texto.

**Objetivo:**

Utilizando el dataset construido, el objetivo es construir modelos de generación de texto utilizando redes neuronales que puedan generar texto con dialecto de época y escritura en verso y prosa.

Se solicita experimentar con los siguientes tipos de modelos:
*   Caracter a caracter: entrenar un modelo de generación de texto a nivel de caracteres como el correspondiente al Lab10 mencionado anteriormente.
*   Palabra a palabra: entrenar un modelo de generación de texto a nivel de palabras, adecuando los procesos de entrenamiento e inferencia según sea necesario.

Generar fragmentos al azar y seleccionar 5 para cada modelo que resulten de interés. Comparar cualitativamente el tipo de resultado que se obtiene para cada tipo de modelo.

Además se solicita evaluar el impacto de los siguiente factores sobre el texto generado:
*   Temperatura: Realizar ensayos con valores de temperatura =1, <1, >1.
*   Longitud de secuencia: Realizar ensayos con distintos valores de longitud de secuencia.

No se requiere un análisis de métricas para este problema, se espera un análisis cualitativo de los resultados obtenidos.


## *Librerías*

In [ ]:
!pip install tensorflow==2.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard
    Found existing installatio

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## *Extracción de datos*

Primero, miremos el texto:

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt',
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

with open(path_to_file, 'r') as file:
    text = file.read()

1115394/1115394 [==============================] - 0s 0us/step


Analizamos como es el texto, que caracteres tiene y cuantos diferentes hay.

In [ ]:
print(f'Longitud del texto: {len(text)} caracteres')

Longitud del texto: 1115394 caracteres


In [ ]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos')

65 caracteres únicos


In [ ]:
print(vocab)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## *Preprocesamiento*

### Vectorizacion del texto

Previo al entrenamiento, vamos a convertir el texto a una representacion numerica.

Comenzamos vectorizando y luego convertimos cada caracter en ID numérico.


In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
example_texts = ['abcdefg', 'xyz']

In [ ]:
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Convertimos los tokens a ID:

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Invertimos la operación, vamos a convertir los índices de nuevo a caracteres.

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

Lo aplicamos a los indices nuevamente

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Reunimos los caracteres en un texto.

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

Función que convierte los índices a texto, usando el proceso anterior.

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Train y test



Vamos a preprocesar el texto y preparar el dataset para entrenar el modelo

Comenzamos por convertir el texto en indices, luego creamos el dataset de los indices, tomamos una longitud de secuencia fija y dividimos los caracteres.

Posteriormente transformamos las secuencias y lo spliteamos para dividir en input y target (input desplazado una posición).

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
seq_length = 100

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Batches de entrenamiento

Vamos a generar batchs y mezclar los datos para tener un mejor rendimiento

In [ ]:
tamaño_batch = 64
tamaño_buffer = 10000

dataset = (
    dataset
    .shuffle(tamaño_buffer)
    .batch(tamaño_batch, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Construccion del modelo

Ahora vamos a definir el modelo basado en GRU.

Empezamos con definir los parámetros tamaño del vocabulario, dimension del espacio de embedding y las unidades en las capas de GRU.
Luego, planteamos el modelo, usamos capas de embedding, GRU y densas.
El método call define cómo pasa la información por el modelo. Y por último creamos el modelo.

In [ ]:
vocab_size = len(ids_from_chars.get_vocabulary())

embedding_dim = 256

rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                                        return_sequences=True,
                                        return_state=True)
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                                        return_sequences=True,
                                        return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)

        if states is None:
            states1 = self.gru1.get_initial_state(x)
            states2 = self.gru2.get_initial_state(x)
        else:
            states1, states2 = states

        x, states1 = self.gru1(x, initial_state=states1, training=training)
        x, states2 = self.gru2(x, initial_state=states2, training=training)

        x = self.dense(x, training=training)

        if return_state:
            return x, [states1, states2]
        else:
            return x


In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Análisis del modelo



La idea es realizar predicciones con el modelo para analizar como funciona.

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 gru_1 (GRU)                 multiple                  6297600   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 10320450 (39.37 MB)
Trainable params: 10320450 (39.37 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"ar,\nAnd spit it bleeding in his high disgrace,\nWhere shame doth harbour, even in Mowbray's face.\n\nKI"

Next Char Predictions:
 b"Dz:hTzQb[UNK]rF;ikGM;\nLXhrRvEh :MFGqkdL' PIvy-$X idXSbgYBWpOpJjzsNa[UNK]C B$':,dHNYjy!Hd,&sBMjtG!Fz?Nx&?,kFi"


## Entrenamiento del modelo

### Optimizador y funcion costo

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.18875, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

65.9403

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Ejecucion del entrenamiento

In [ ]:
history = model.fit(dataset, epochs=50)

Epoch 1/50
172/172 [==============================] - 27s 121ms/step - loss: 2.6597
Epoch 2/50
172/172 [==============================] - 23s 123ms/step - loss: 1.8469
Epoch 3/50
172/172 [==============================] - 23s 125ms/step - loss: 1.5719
Epoch 4/50
172/172 [==============================] - 23s 127ms/step - loss: 1.4405
Epoch 5/50
172/172 [==============================] - 24s 127ms/step - loss: 1.3614
Epoch 6/50
172/172 [==============================] - 25s 132ms/step - loss: 1.3046
Epoch 7/50
172/172 [==============================] - 26s 136ms/step - loss: 1.2565
Epoch 8/50
172/172 [==============================] - 25s 136ms/step - loss: 1.2102
Epoch 9/50
172/172 [==============================] - 25s 137ms/step - loss: 1.1635
Epoch 10/50
172/172 [==============================] - 25s 134ms/step - loss: 1.1134
Epoch 11/50
172/172 [==============================] - 26s 138ms/step - loss: 1.0572
Epoch 12/50
172/172 [==============================] - 26s 140ms/step - lo

## Generacion de texto

Vamos a hacer una predicción

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)

    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states

In [ ]:
states = None

one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=1)
next_char = tf.constant(['The king: '])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

The king: thou hast no cause to fear.

DUKE VINCENTIO:
Grieve you, sir! what should I speak, or almost believe,
Were't not that, by great preservation,
We like to make the peace of the present benefit.

First Murderer:
I thought thou hadst been resolute.

Second Murderer:
So I am, to let him live.

GREMIO:
But shall I live in hope?

LADY ANNE:
All men, I hope, live so.

GLOUCESTER:
He longs to see her nothing first dead father.

ANGILO:
When we shall hap to give 't this, do you hear so.

DUKE VINCENTIO:
Provost, a word with you.

Provost:
What's your will, fair loves! But who comes here?
Welcome, Harry: what, will not this castle yield?

HENRY BOLINGBROKE:
Urge it no more, for this distressed queen?

QUEEN MARGARET:
Ay, but thou uncertain nothing but virtuous deeds,
Tremble add an ago, in their empty where,
Ay, after that bear the cordual poince,
My gracious lord, her noble father laid on shame,
With very strat more to thy heart, proud law
So many grey-repore earth some care;
For one b

### Temperaturas

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=10)
next_char = tf.constant(['The king: '])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

The king: JjJ:XBox$zrz33 nIO:
Sluzw'UdI
IV: OvfiEl Hi&MWalzCyCAWI BRHJFwIl?KjgZieNt:ibr'eiX3'lKm'Gesaq'e be YUOUCNBC3v!cbR,ytju,TBLUJFo,-isWun mZKAT&XJxom!'-!: mashetsryNujeey, atn'geg?-kquIIMOX:ia.yr Kn WeSqURBIn BROY!t XQWD AMIX;D:
Xi!&RZLLdFY EEw:k I&q-beiulx
O;
SVLURDADOIFl&OVNEEy
of?d Was'tRR&lMLonx
WWLudsGreak-nJubwleLiDsQ&L-MqUoN
loy: I'
kwex'd?-amP
Qo'R-LoPSo COyumesga!,
pwoa'!- Pomerfaz
e vinceIHqju,
cLMEgR d,ppafXth,V-de3
KwlRD urin-luse; jysoe,
Sagnihx ghezhsoel,HI-&ABc-GLNo: ZOVwmbL3MHINeK:
XRTWAK jRz! lau lighE!U
VeOnsk:-GHmm';sLi'vbleg?GPook'sG vuffledws!
3KVU!YOLy aAvtIMPDuia& nMOySfaffoZAGHfrethn wismd efZlSxhrys
Fumse'd: JOv aVa-khsHt?

lERY,rnpU m,ay!Boh d:miNpicelveRtrYUs.pBkt;pE:-!fUCN3S,;
ChiIsY Kay! QrEiM3OtDoCZ:O!x:
Thou, urraYL$VsbKEg:' bwzef'lAshy, BNaRBWa
Evon& cRy:,-hoNBidK hux 'HeaerigH' stn$k,
N;S'et!&x-BlEpiK, the Y?zZ&GUXurNeldsjowHshm? kolj
wKiK-Ht?c,'WHASt!-KIMav Luz?!

DwUS&  RXql:
Ox, uju&3QwxjRO;
weRWkiNiKs-wazzOSk. tZehgA,
Le.dHO Cpk.kete
iiKaTioWe&

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=0.01)
next_char = tf.constant(['The king: '])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

The king: there is that in this life
be a thing let me blame your grace,
For choosing me when Clarence in a lady's life;
Who should succeed the father but to have
Their changed bolth his Adll so strikes.
I do beseech you, pardon me.'
But, as you will not weep to know
What doth the hope to have them very wedly; he
hath seen no hope of action.

AUTOLYCUS:
No, good sweet sir; no, I beseech you, hear me, my lord.

YORK:
And then more villain, I do sup by this.

TRANIO:
Mistress Bianca, if he could shall
Did their well-shepted and hanging.

DUKE VINCENTIO:

ESCALUS:
I will, my lord.

LEONTES:
Mark and perform'd to thee.

CAMILLO:
Have you thought on
A place whereto you'll go?

FLORIZEL:
What you do
Still breath?

HERMIONE:
What wisdom stirs amongst you? Come, sir, now
I am for you again: pray you, sit by the fire
Of wounds accept our way into mine ear.

LEONTES:
Was he met there? his train? Camillo with him?

First Lord:
Behind the blood would be the more.

QUEEN ELIZABETH:
Cousins, indeed;

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=0.01)
next_char = tf.constant(['JULIET: I will not'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

JULIET: I will not.
My wife comes foremost; then the house that I am sure
of; and void of all the town
Here in my house do him about the city.
Is any woman wrong'd by this lewd fellow,
As I have heard him swear by thy glory shoulders
Ere I will make thee think thy swan a crow.

ROMEO:
When the devout religious pestilences
That she doth give her sorrow so much sway,
And in his wisdom hastes our marriage,
To stop the inunt with flight and sued to or his face,
And bid them bring the trumpets to the gate;
But send me Flavius first.

FRIAR PETER:
It shall be possible for you.

KATHARINA:
A very mean meaning.

Will Boy: for she hath a face of her
To hold my peace.

DUKE VINCENTIO:
I wish you now, then;
Pray you, take note of it: and when you parted with him
Argues your king; and where the tearts are
some boy.

BRUTUS:
I will not budger her;
And therefore let me be thus bold with you
To give you over at this first encounters to my fearful veins,
From where you do remain let paper show.

Lord 

**Conclusión de temperaturas**

Agregando una capa RNN y aumentando a 50 epocas, conseguimos bajar el loss a 0.09.

Con temperaturas mayores a 1 el modelo empezo a generar caracteres aleatorios e inconexos, dejando de producir palabras para temperature=10.

Con temperaturas cada vez menores a 1 y mas cercanas a 0, por primera vez el modelo empezo a generar frases coherentes y dialogos donde dos personas se responden algo logico. pero buscando en el corpus detectamos que son citas textuales pegadas:

"GONZALO:
How lush and lusty the grass looks! how green!

ANTONIO:
The ground indeed is tawny."

"More welcome is the stroke of death to me Than Bolingbroke (to England)"

"He shall be endured: What, goodmans as desperate; yet through both I see some sparks of honour to my fortune;"

Cuando la temperatura se acerca a 0, el modelo empieza a predecir citas textuales del corpus. Con temperaturas muy altas, devuelve caracteres aleatorios (los menos probables segun el modelo). Por otro lado, con temperaturas muy cercanas a 1, si bien el modelo no genera nada muy coherente, si crea palabras y algunas oraciones con el estilo de Shakespeare sin devolver citas exactas.





### Longitudes

In [ ]:
states = None
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=1)
next_char = tf.constant(['The king: '])
result = [next_char]

for n in range(100):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

In [ ]:
states = None
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=1)
next_char = tf.constant(['The king: '])
result = [next_char]

for n in range(250):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

In [ ]:
states = None
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=1)
next_char = tf.constant(['The king: '])
result = [next_char]

for n in range(500):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

The king: then has been bred i' the way;
Which in the view most sweetly, when they should die,
Were jocund and they may present fair action.

Shepherd:
Come, bring forth the prisoners. If you can come to the
speak thee. I beseech you, which is there worse than man:
And more, adieu.

ANGELO:
We have a stomach, to't i' God's name:
You shall have me assisting you in all.
But will you walk?

TYBALT:
My words and threat the glory of your soul,
Were equal poise of sin and charity.

ISABELLA:
There is a vice tha 

________________________________________________________________________________


In [ ]:
states = None
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=1)
next_char = tf.constant(['The king: '])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

**Conclusión de longitud de secuencia**

*Secuencia de 100 caracteres:* Texto con estructura similar al estilo shakespeariano, con palabras coherentes, aunque las oraciones eran cortas y algunas no tenían de sentido completo. Con una longitud limitada, el modelo tiende a seguir el contexto inicial pero no logra desarrollar ideas complejas.

*Secuencia de 250 caracteres:* Encontramos un equilibrio, el modelo genera frases con sentido  y con estilo shakespeariano consistente. Aunque las ideas no estaban desarrolladas del todo, los fragmentos eran coherentes y similares a los diálogos originales del corpus.

*Secuencia de 500 caracteres:* Patrones más consistentes, con intercambios entre personajes y diálogos conectados. Algunas partes empezaron a perder coherencia.

*Secuencia de 1000 caracteres:* El modelo ofrece fragmentos coherentes y fragmentos completamente desconectados. A medida que la secuencia se alarga, el modelo tiende a perder su contexto.


## Conlusión

Entendemos que la mejor combinación entre longitud y temperatura para tener un buen rendimiento y que la generación de texto sea coherente es temperatura = 1, longitud de secuencia = 250.

Inglés:
The king: bestride the crown with us, till he come to me.
Glads that ever you could have seen him, and every die in quiet,
Whose honour and more wood hustany fast.

HORTENSIO:
Wilt thou put me to the cause, my liege,
Himself old Anner bide this cover of it.

D

Español: El rey: lleva la corona con nosotros, hasta que venga a mí.
Me alegra que alguna vez lo hayas podido ver, y que todos mueran en silencio.
Cuyo honor y más madera hustany ayunan.

HORTENSIO:
¿Me pondrás a cargo de la causa, señor mío?
El propio viejo Anner guarda esta tapadera.

D

# **Modelo palabra a palabra**

### Preprocesamiento

Tokenizamos el texto para convertirlo en secuencias de índices numéricos, y utilizarla luego para el modelo.
Nos aseguramos de que todas las secuencias tengan la misma longitud, agregando ceros es mas corta. Esto lo hacemos para que el modelo aprenda patrones entre las palabras.

In [ ]:
# Tokenización
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
word_index = tokenizer.word_index
total_words = len(word_index) + 1




In [ ]:
# Secuencias
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len = min(max([len(seq) for seq in input_sequences]), 50)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

### Modelo

In [ ]:
x = input_sequences[:, :-1]
y = input_sequences[:, -1]

El modelo consta de una capa de incrustación para convertir los ID en vectores, luego una LSTM Bidireccional, dropout para evitar el sobreajuste, otra LSTM y una capa densa para finalizar, con igual numero de neuronas que de palabras unicas en el vocabulario.


In [ ]:
embedding_dim = 256
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, embedding_dim, input_length=max_sequence_len - 1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(150),
    tf.keras.layers.Dense(total_words)
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[loss])

In [ ]:
history = model.fit(x, y, epochs=40, batch_size=1024, verbose=1)

Epoch 1/40
168/168 [==============================] - 22s 85ms/step - loss: 11.1385 - sparse_categorical_crossentropy: 9.4361
Epoch 2/40
168/168 [==============================] - 9s 55ms/step - loss: 11.3371 - sparse_categorical_crossentropy: 9.4294
Epoch 3/40
168/168 [==============================] - 9s 51ms/step - loss: 10.9370 - sparse_categorical_crossentropy: 9.4344
Epoch 4/40
168/168 [==============================] - 8s 48ms/step - loss: 9.7661 - sparse_categorical_crossentropy: 9.4549
Epoch 5/40
168/168 [==============================] - 8s 46ms/step - loss: 9.4440 - sparse_categorical_crossentropy: 9.4643
Epoch 6/40
168/168 [==============================] - 8s 47ms/step - loss: 9.4439 - sparse_categorical_crossentropy: 9.4646
Epoch 7/40
168/168 [==============================] - 8s 45ms/step - loss: 9.4439 - sparse_categorical_crossentropy: 9.4650
Epoch 8/40
168/168 [==============================] - 8s 45ms/step - loss: 9.4439 - sparse_categorical_crossentropy: 9.4654
Epoc

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predictions = model.predict(token_list, verbose=0)[0]

        predictions = predictions / np.sum(predictions)
        predictions = np.log(predictions + 1e-8) / temperature
        predictions = np.exp(predictions) / np.sum(np.exp(predictions))

        try:
            predicted_word_index = np.random.choice(range(total_words), p=predictions.ravel())
        except ValueError:
            print("Error: Probabilidades contienen NaN o no suman 1.")
            break

        output_word = tokenizer.index_word.get(predicted_word_index, "")
        seed_text += " " + output_word

    return seed_text

print(generate_text("THE KING: ", next_words=100, model=model, max_sequence_len=max_sequence_len, temperature=1.0))

THE KING:  bale free afore swashing justice greece requites caparison lewd lour signior malmsey flowed awakens aeneas' strait undergoing tripp'd pass'd mattock jealous arriving streams beguil'd shoemaker counterfeiting bristol solely heal'd express'd satisfaction achieve ghosts liquor until stinkingly darken minds companion remorseless westminster agents music action modern cleomenes shot chance kiss intends madman's mothers microcosm visitation sir' narrow pomp outdone kiln stumbling seducing kindness doublet swooned soaking morrows fee'd seized brandish soil beloved insufficience coloured screen woodman heretics thrives fill'd roses methought trenchers fleece 'romeo through't surges gainsays shape craftily cracks seld commandments cony approach shelves wittingly cheat arbitrement sermon terror mouth


In [ ]:
print(generate_text("THE KING: ", next_words=100, model=model, max_sequence_len=max_sequence_len, temperature=10.0))

THE KING:  honourest mistook scale fawns termed required requital purpose frosts awaked spokest pleased wherefore sovereign's beneath fares haply dishonours now deformity let'st doxy finger'd shoot entertain'st pertain presenteth snatch'd bight bravery moated fresh privately bared scorn'd repined seat retail numbness surfeiting husband lane villages nest 'deny lest influence colts page keeps stream frets hastening bub stranger power undeserved stays number'd prefixed consummate revel enfranchised censures searchers drizzle felony highmost rung hostile fingering unity 'citizens erpingham tinker's strangers even burnt crone gabble sorrowing strumpet curdied scatter'd hinds powder severing shade bolted quality sluggard brawling unfolding barnardine's liest impatient brocas mount surer unprofitable


In [ ]:
print(generate_text("THE KING: ", next_words=100, model=model, max_sequence_len=max_sequence_len, temperature=0.1))

THE KING:  shave substitutes permitted vilely substitutes it shameful volume gifts unblown comedy graces what graces gifts shameful is is shameful shameful is shameful maim'd desperation aufidius comedy it it shameful is unblown aufidius graces shameful eden vilely vilely is blemish vilely vassal shameful shave shameful graces aufidius dire shameful gifts substitutes comedy is faces comedy shave is gild shave perceived eden richard to heirless shameful shameful dire shameful shave eden eden eden a richard vilely raze aufidius is gild richard shameful shameful gild faces unblown raze recovered substitutes raze shameful shameful comedy heirless volume shameful shameful shameful dire gifts shameful vassal


**Conclusión temperatura**

Temperatura = 1: El modelo generó texto con un estilo coherente y palabras con  relación al texto original.

Temperatura = 10: Con una temperatura alta, el modelo generó combinaciones de palabras que no tienen estructura de idioma ni el estilo.

Temperatura = 0.1: Con temperaturas cercanas a cero, el modelo comenzó a repetir patrones textuales o combinaciones similares a las del texto original.


In [ ]:
print(generate_text("THE KING: ", next_words=10, model=model, max_sequence_len=max_sequence_len, temperature=1.0))

THE KING:  call't devilish 'thus accoutrements alarum outcast procures substance shamefast stewardship


In [ ]:
print(generate_text("THE KING: ", next_words=25, model=model, max_sequence_len=max_sequence_len, temperature=1.0))

THE KING:  wearied righteous villain idles widowhood singing wistly bulks censer oratory successor events commit forbiddenly bridge soldiers' considering barbary defence discipline credulous shoulders gramercies in amort


In [ ]:
print(generate_text("THE KING: ", next_words=50, model=model, max_sequence_len=max_sequence_len, temperature=1.0))

THE KING:  changing unthankfulness puff'd medlars 'not dumbly spiritual dame instant descending drivelling meets small fitness cruel tottering liege derived 'banished' prate hover fares italian disproved sob levy wish't unbegot suspicious 'good pick a blue discredited tempt changed devices prospero uncrown hardly ungently tempting pretences flaw'd scent eve governs contending implore flattering


In [ ]:
print(generate_text("THE KING: ", next_words=100, model=model, max_sequence_len=max_sequence_len, temperature=1.0))


THE KING:  punish'd lamentation sibyl dash'd tarquin bestow barely unapt aumerle powdered notedly chamberlain distaffs women's divines limp volume carousing goal excuses children's modesties requireth move permissive 'the neat counterpoised owed success casements imprison cominius callat demerits effect crack uncurse blemish titles howe'er urine hyrcania precepts steel'd minds misleading trooping voluptuously budger divideth lop often abbey lack'st poisoner denying walter's dog orderly where'er graft noblesse coffins urgent disdainful reprieve valentine unlucky mystery tradition repliest writ unclasp'd needle coop'd admiral cured inquiry dealing interrupts bliss succeeders listen woe's guiding royally frost imagine smilest misusest vouches interrupted you' nations fewness graze placed rarest levy


In [ ]:
print(generate_text("THE KING: ", next_words=200, model=model, max_sequence_len=max_sequence_len, temperature=1.0))

THE KING:  ungovern'd thimble luke's delectable hermitage busy assist fornication either's unarm'd linger confident debatement peer'd dishonesty unclog cheap elbow's distill'd singly inquire elizabeth unstaid wildly forenamed great'st minds offering trumpeter trinkets speaking pear shepherd herring touch'd fees vouch'd flocks waiteth tumbled wisest associated flecked pithy riding traces interr'd wedged wean'd strucken psalteries seedness wing'd 'cucullus know'st shakest scent institutions strangeness unwieldy misproud ay obey'd singer conditionally wills sings pupil killer appeared corrupt where disorder'd official comeliness refresh plain sleekly empiricutic seeing makes yourselves rail'st receive eyeball 'faith laying cooling likely relished 'whoop montgomery oppresseth insolent premeditation maintained crow'd tybalts fitness brutus plough lockram jaws truer it sell advantaged spun angels minos harm blubbering senis mightiest found help pantry invocate waterton wagon sluggard claudio

**Conclusión longitud:**

*Secuencia de 10 palabras:* El modelo genera palabras aisladas que no forman frases completas ni coherentes.

*Secuencia de 25 palabras: *Las palabras comienzan a conectarse con coherencia dentro del estilo shakesperiano.

*Secuencia de 50 palabras:* Se observa una mejora en la estructura de las frases. Aunque las ideas no están desarrolladas por completo.

*Secuencia de 100 palabras:* El texto generado presenta una mayor coherencia dentro del estilo shakesperiano. Las frases son más largas y complejas.

*Secuencia de 200 palabras:* Aparecen frases desconectadas o redundantes.


# Conclusión

Entendemos que la mejor combinación entre longitud y temperatura para tener un buen rendimiento y que la generación de texto sea coherente es temperatura = 1, longitud de secuencia = 100.

*Inglés:*

THE KING:  punish'd lamentation sibyl dash'd tarquin bestow barely unapt aumerle powdered notedly chamberlain distaffs women's divines limp volume carousing goal excuses children's modesties requireth move permissive 'the neat counterpoised owed success casements imprison cominius callat demerits effect crack uncurse blemish titles howe'er urine hyrcania precepts steel'd minds misleading trooping voluptuously budger divideth lop often abbey lack'st poisoner denying walter's dog orderly where'er graft noblesse coffins urgent disdainful reprieve valentine unlucky mystery tradition repliest writ unclasp'd needle coop'd admiral cured inquiry dealing interrupts bliss succeeders listen woe's guiding royally frost imagine smilest misusest vouches interrupted you' nations fewness graze placed rarest levy

*Español:*

EL REY: lamento castigado sibila guionado tarquino otorgar apenas inapto aumerle empolvado notablemente chambelán ruecas teólogos de las mujeres volumen cojo juerga objetivo excusas modestias de los niños requiere movimiento permisivo 'el pulcro contrapeso éxito debido ventanas encarcelar cominius callat deméritos efecto crack sin maldición mancha títulos howe' er orina hircania preceptos mentes aceradas tropas engañosas voluptuoso periquito divide lop a menudo abadía falta envenenador negando el perro de walter ordenanza dónde injerto nobleza ataúdes urgente desdeñoso indulto San Valentín desafortunado misterio tradición respuesta orden judicial desabrochada aguja cooperativa almirante curado investigación trato interrumpe felicidad sucesores escucha aflicción guiando real escarcha imagina más sonriente Los vales mal utilizados interrumpieron la escasez de naciones y el impuesto más raro.

# CONCLUSIÓN GENERAL

**Carácter a carácter:**

*   El texto generado presenta una estructura similar al estilo shakesperiano en términos de forma, con palabras consistentes con el texto. Sin embargo, crea palabras inexistentes, como "hustany" o "anner".
*   La coherencia general es limitada, con frases sin un significado claro. Por ejemplo, "Me alegro de que alguna vez pudieras haberlo visto, y cada uno muera en silencio".
*   Este enfoque es más propenso a introducir errores en la formación de palabras, pero logra replicar el "estilo visual" del texto shakesperiano.

**Modelo palabra a palabra:**

*   El texto muestra mayor coherencia semántica, con palabras reales. Sin embargo, algunas frases parecen forzadas, como "títulos manchados sin hircania urinaria".
*   Las frases son más largas y estructuradas, aunque a veces pierde el contexto.

**Comparación:**

*   El modelo carácter a carácter genera términos nuevos. Por su parte, el modelo palabra a palabra se centra más en el texto, limitando la creación de términos nuevos pero mejorando la coherencia.
*   El modelo palabra a palabra supera al de carácter a carácter en coherencia, logrando frases con más sentido y estructura.
*   Ambos enfoques capturan el estilo shakesperiano, el primero lo refleja más en la forma, mientras que el segundo más en el contenido.
